In [ ]:
import mne
import os
import numpy as np
import warnings
from mne.stats import fdr_correction, f_mway_rm
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline

#############################################
# 1. Data Preprocessing and Loading
#############################################
# Define the planned time range and baseline correction time range
desired_tmin = -2.1      # Start time of the cropped data
desired_tmax = -0.2      # End time of the cropped data
baseline_correct_tmin =   # Baseline correction start time
baseline_correct_tmax =    # Baseline correction end time

def read_raw(filename: str) -> mne.io.Raw:
    print(filename)
    if filename.endswith('.fif'):
        raw = mne.io.read_raw_fif(filename, preload=True)
    else:
        raw = mne.io.read_raw_eeglab(filename, preload=True)
    return raw

# Define data types and paths
stimuli_type = ['Reward_Cases', 'Punish_Cases', 'Reward_Avatar', 'Punish_Avatar']
ascent_air = 'Test_Epochs_Approach/'

# Define the path for saving images
output_dir = 'ERP/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# View files in the Reward_Cases folder
for item in os.listdir(os.path.join(ascent_air, 'Reward_Cases')):
    if item.endswith('.fif'):
        print(item, end=' ')
print("\n")

def link_participants_data(data_type: str):
    nlist = []
    nlist_ndarray = []
    folder_path = os.path.join(ascent_air, data_type)
    
    for item in os.listdir(folder_path):
        if item.endswith('.fif'):
            print(f"Processing file: {item}")
            epochs_fname = os.path.join(folder_path, item)
            epochs = mne.read_epochs(epochs_fname, preload=True)
            
            print("Initial epochs data shape:", epochs.get_data().shape)
            epochs.apply_baseline(baseline=(baseline_correct_tmin, baseline_correct_tmax))
            print("Epochs data shape after baseline correction:", epochs.get_data().shape)
            
            evoked = epochs.average()
            print("Evoked data shape after averaging:", evoked.data.shape)
            
            evoked.crop(tmin=desired_tmin, tmax=desired_tmax)
            print("Evoked data shape after cropping:", evoked.data.shape, "\n")
            
            nlist.append(evoked)
            nlist_ndarray.append(evoked.get_data())
    
    return nlist, nlist_ndarray

# Load data for each condition
R_Case_list, R_Case_list_ndarray = link_participants_data('Reward_Cases')
P_Case_list, P_Case_list_ndarray = link_participants_data('Punish_Cases')
R_Avatar_list, R_Avatar_list_ndarray = link_participants_data('Reward_Avatar')
P_Avatar_list, P_Avatar_list_ndarray = link_participants_data('Punish_Avatar')

print("Number of participants per condition:", len(R_Case_list), len(P_Case_list), len(R_Avatar_list), len(P_Avatar_list))
print("Example data shapes:", R_Case_list_ndarray[0].shape, P_Case_list_ndarray[0].shape, 
      R_Avatar_list_ndarray[0].shape, P_Avatar_list_ndarray[0].shape)

# Calculate grand average (group ERP) for each condition
R_Case_Merged = mne.grand_average(R_Case_list)
P_Case_Merged = mne.grand_average(P_Case_list)
R_Avatar_Merged = mne.grand_average(R_Avatar_list)
P_Avatar_Merged = mne.grand_average(P_Avatar_list)

# Use a unified time axis from one evoked object
times = R_Case_Merged.times
print("Unified time axis shape:", times.shape)

#############################################
# 2. Final 2×2 ANOVA Plot: Statistical Analysis and Plotting with Unified Time Axis
#############################################
# Use grand average data
data_A1B1 = R_Case_Merged.data  # Space Navigation Reward
data_A1B2 = P_Case_Merged.data  # Space Navigation Punishment
data_A2B1 = R_Avatar_Merged.data  # Social Navigation Reward
data_A2B2 = P_Avatar_Merged.data  # Social Navigation Punishment

# Reshape data to (n_channels, 1, n_times)
reshaped_A1B1 = data_A1B1.reshape(data_A1B1.shape[0], 1, -1)
reshaped_A1B2 = data_A1B2.reshape(data_A1B2.shape[0], 1, -1)
reshaped_A2B1 = data_A2B1.reshape(data_A2B1.shape[0], 1, -1)
reshaped_A2B2 = data_A2B2.reshape(data_A2B2.shape[0], 1, -1)

# Combine data by condition: resulting shape (n_channels, 4, n_times)
data_combine = np.concatenate((reshaped_A1B1, reshaped_A1B2, reshaped_A2B1, reshaped_A2B2), axis=1)
factor_levels = [2, 2]

# Perform 2×2 repeated measures ANOVA (for each time point)
f_main_A, p_main_A = f_mway_rm(data_combine, factor_levels, effects='A')
f_main_B, p_main_B = f_mway_rm(data_combine, factor_levels, effects='B')
f_inter, p_interaction = f_mway_rm(data_combine, factor_levels, effects='A:B')

# FDR correction
rejects_A, p_main_A = fdr_correction(p_main_A, alpha=0.05)
rejects_B, p_main_B = fdr_correction(p_main_B, alpha=0.05)
rejects_inter, p_interaction = fdr_correction(p_interaction, alpha=0.05)

# Plot the final 2×2 ANOVA figure
plt.figure(figsize=(10, 6))
plt.plot(times, np.average(data_A1B1, axis=0), label='A1B1 (Space Navigation Reward)', color='blue')
plt.plot(times, np.average(data_A1B2, axis=0), label='A1B2 (Space Navigation Punishment)', color='green')
plt.plot(times, np.average(data_A2B1, axis=0), label='A2B1 (Social Navigation Reward)', color='orange')
plt.plot(times, np.average(data_A2B2, axis=0), label='A2B2 (Social Navigation Punishment)', color='purple')

# Add vertical lines at significant time points (different colors for different effects)
for i in range(len(times)):
    if p_main_A[i] < 0.05:
        plt.axvline(x=times[i], ymin=0.01, ymax=0.06, color='grey', alpha=0.5, linestyle='--',
                    label='Main Effect A' if i == 0 else "")
    if p_main_B[i] < 0.05:
        plt.axvline(x=times[i], ymin=0.07, ymax=0.12, color='blue', alpha=0.5, linestyle='--',
                    label='Main Effect B' if i == 0 else "")
    if p_interaction[i] < 0.05:
        plt.axvline(x=times[i], ymin=0.13, ymax=0.18, color='red', alpha=0.5, linestyle='--',
                    label='Interaction Effect' if i == 0 else "")

plt.title('ERP Analysis with Two-Way ANOVA')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (µV)')
plt.legend(loc='best')
plt.tight_layout()
plt.grid(alpha=0.3)

output_path = os.path.join(output_dir, 'ERP_Two_Way_ANOVA.png')
plt.savefig(output_path, dpi=300)
print(f"Figure saved to {output_path}")
plt.show()

#############################################
# 3. Combined Evoked Data Comparison Plot (Retain This Plot)
#############################################
# Use evoked results from real data
merged_evoke = {
    'Space_Navigation_Reward': R_Case_Merged,
    'Space_Navigation_Punishment': P_Case_Merged,
    'Social_Navigation_Reward': R_Avatar_Merged,
    'Social_Navigation_Punishment': P_Avatar_Merged
}

def plot_evoked_with_anova(merged_evoke, picks, title, output_name):
    # Convert channel labels to indices using channel names from one evoked object
    ch_names = next(iter(merged_evoke.values())).ch_names
    picks_idx = mne.pick_channels(ch_names, picks)
    
    # Get data for selected channels
    data_list = [evk.data[picks_idx] for evk in merged_evoke.values()]
    
    # Reshape data to (n_channels, 1, n_times)
    reshaped_data = [data.reshape(data.shape[0], 1, -1) for data in data_list]
    
    # Combine data (shape: n_channels, 4, n_times)
    data_combine = np.concatenate(reshaped_data, axis=1)
    
    # Perform 2×2 repeated measures ANOVA (for each time point)
    f_main_A, p_main_A = f_mway_rm(data_combine, factor_levels, effects='A')
    f_main_B, p_main_B = f_mway_rm(data_combine, factor_levels, effects='B')
    f_inter, p_interaction = f_mway_rm(data_combine, factor_levels, effects='A:B')

    # FDR correction
    rejects_A, p_main_A = fdr_correction(p_main_A, alpha=0.05)
    rejects_B, p_main_B = fdr_correction(p_main_B, alpha=0.05)
    rejects_inter, p_interaction = fdr_correction(p_interaction, alpha=0.05)

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))
    mne.viz.plot_compare_evokeds(evokeds=merged_evoke, picks=picks_idx, combine='mean', axes=ax, legend='lower right')
    
    # Add vertical lines at significant time points (using ax instead of plt)
    for i in range(len(times)):
        if p_main_A[i] < 0.05:
            ax.axvline(x=times[i], ymin=0.01, ymax=0.06, color='grey', alpha=0.5, linestyle='--',
                       label='Main Effect A' if i == 0 else "")
        if p_main_B[i] < 0.05:
            ax.axvline(x=times[i], ymin=0.07, ymax=0.12, color='blue', alpha=0.5, linestyle='--',
                       label='Main Effect B' if i == 0 else "")
        if p_interaction[i] < 0.05:
            ax.axvline(x=times[i], ymin=0.13, ymax=0.18, color='red', alpha=0.5, linestyle='--',
                       label='Interaction Effect' if i == 0 else "")
    
    fig.suptitle(title)
    fig.savefig(os.path.join(output_dir, output_name), dpi=300)
    print(f"Figure saved to {os.path.join(output_dir, output_name)}")


# Plot ANOVA figures for different regions and save
plot_evoked_with_anova(merged_evoke, picks=['F3', 'F7', 'FC5'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Left Frontal Cortex. Timestamp: -2.1s~0s',
                       output_name='lFC_ERP_ANOVA.png')

plot_evoked_with_anova(merged_evoke, picks=['F4', 'F8', 'FC6'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Right Frontal Cortex. Timestamp: -2.1s~0s',
                       output_name='rFC_ERP_ANOVA.png')

plot_evoked_with_anova(merged_evoke, picks=['CP5', 'P7', 'P3'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Left Temporoparietal Junction (lTPJ). Timestamp: -2.1s~0s',
                       output_name='lTPJ_ERP_ANOVA.png')

plot_evoked_with_anova(merged_evoke, picks=['CP6', 'P4', 'P8'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Right Temporoparietal Junction (rTPJ). Timestamp: -2.1s~0s',
                       output_name='rTPJ_ERP_ANOVA.png')

plt.show()

In [ ]:
import mne
import os
import numpy as np
import warnings
from mne.stats import fdr_correction, f_mway_rm
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline

#############################################
# 1. Data Preprocessing and Loading
#############################################
# Define the planned time range and baseline correction time range
desired_tmin = 0       # Start time of the cropped data
desired_tmax = 4    # End time of the cropped data
baseline_correct_tmin =   # Baseline correction start time
baseline_correct_tmax =   # Baseline correction end time

def read_raw(filename: str) -> mne.io.Raw:
    print(filename)
    if filename.endswith('.fif'):
        raw = mne.io.read_raw_fif(filename, preload=True)
    else:
        raw = mne.io.read_raw_eeglab(filename, preload=True)
    return raw

# Define data types and paths
stimuli_type = ['Reward_Cases', 'Punish_Cases', 'Reward_Avatar', 'Punish_Avatar']
ascent_air = 'Test_Epochs_Asred/'

# Define the path for saving images
output_dir = 'ERP/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# View files in the Reward_Cases folder
for item in os.listdir(os.path.join(ascent_air, 'Reward_Cases')):
    if item.endswith('.fif'):
        print(item, end=' ')
print("\n")

def link_participants_data(data_type: str):
    nlist = []
    nlist_ndarray = []
    folder_path = os.path.join(ascent_air, data_type)
    
    for item in os.listdir(folder_path):
        if item.endswith('.fif'):
            print(f"Processing file: {item}")
            epochs_fname = os.path.join(folder_path, item)
            epochs = mne.read_epochs(epochs_fname, preload=True)
            
            print("Initial epochs data shape:", epochs.get_data().shape)
            epochs.apply_baseline(baseline=(baseline_correct_tmin, baseline_correct_tmax))
            print("Epochs data shape after baseline correction:", epochs.get_data().shape)
            
            evoked = epochs.average()
            print("Evoked data shape after averaging:", evoked.data.shape)
            
            evoked.crop(tmin=desired_tmin, tmax=desired_tmax)
            print("Evoked data shape after cropping:", evoked.data.shape, "\n")
            
            nlist.append(evoked)
            nlist_ndarray.append(evoked.get_data())
    
    return nlist, nlist_ndarray

# Load data for each condition
R_Case_list, R_Case_list_ndarray = link_participants_data('Reward_Cases')
P_Case_list, P_Case_list_ndarray = link_participants_data('Punish_Cases')
R_Avatar_list, R_Avatar_list_ndarray = link_participants_data('Reward_Avatar')
P_Avatar_list, P_Avatar_list_ndarray = link_participants_data('Punish_Avatar')

print("Number of participants per condition:", len(R_Case_list), len(P_Case_list), len(R_Avatar_list), len(P_Avatar_list))
print("Example data shapes:", R_Case_list_ndarray[0].shape, P_Case_list_ndarray[0].shape, 
      R_Avatar_list_ndarray[0].shape, P_Avatar_list_ndarray[0].shape)

# Calculate grand average (group ERP) for each condition
R_Case_Merged = mne.grand_average(R_Case_list)
P_Case_Merged = mne.grand_average(P_Case_list)
R_Avatar_Merged = mne.grand_average(R_Avatar_list)
P_Avatar_Merged = mne.grand_average(P_Avatar_list)

# Use a unified time axis from one evoked object
times = R_Case_Merged.times
print("Unified time axis shape:", times.shape)

#############################################
# 2. Final 2×2 ANOVA Plot: Statistical Analysis and Plotting with Unified Time Axis
#############################################
# Use grand average data
data_A1B1 = R_Case_Merged.data  # Space Navigation Reward
data_A1B2 = P_Case_Merged.data  # Space Navigation Punishment
data_A2B1 = R_Avatar_Merged.data  # Social Navigation Reward
data_A2B2 = P_Avatar_Merged.data  # Social Navigation Punishment

# Reshape data to (n_channels, 1, n_times)
reshaped_A1B1 = data_A1B1.reshape(data_A1B1.shape[0], 1, -1)
reshaped_A1B2 = data_A1B2.reshape(data_A1B2.shape[0], 1, -1)
reshaped_A2B1 = data_A2B1.reshape(data_A2B1.shape[0], 1, -1)
reshaped_A2B2 = data_A2B2.reshape(data_A2B2.shape[0], 1, -1)

# Combine data by condition: resulting shape (n_channels, 4, n_times)
data_combine = np.concatenate((reshaped_A1B1, reshaped_A1B2, reshaped_A2B1, reshaped_A2B2), axis=1)
factor_levels = [2, 2]

# Perform 2×2 repeated measures ANOVA (for each time point)
f_main_A, p_main_A = f_mway_rm(data_combine, factor_levels, effects='A')
f_main_B, p_main_B = f_mway_rm(data_combine, factor_levels, effects='B')
f_inter, p_interaction = f_mway_rm(data_combine, factor_levels, effects='A:B')

# FDR correction
rejects_A, p_main_A = fdr_correction(p_main_A, alpha=0.05)
rejects_B, p_main_B = fdr_correction(p_main_B, alpha=0.05)
rejects_inter, p_interaction = fdr_correction(p_interaction, alpha=0.05)

# Plot the final 2×2 ANOVA figure
plt.figure(figsize=(10, 6))
plt.plot(times, np.average(data_A1B1, axis=0), label='A1B1 (Space Navigation Reward)', color='blue')
plt.plot(times, np.average(data_A1B2, axis=0), label='A1B2 (Space Navigation Punishment)', color='green')
plt.plot(times, np.average(data_A2B1, axis=0), label='A2B1 (Social Navigation Reward)', color='orange')
plt.plot(times, np.average(data_A2B2, axis=0), label='A2B2 (Social Navigation Punishment)', color='purple')

# Add vertical lines at significant time points (different colors for different effects)
for i in range(len(times)):
    if p_main_A[i] < 0.05:
        plt.axvline(x=times[i], ymin=0.01, ymax=0.06, color='grey', alpha=0.5, linestyle='--',
                    label='Main Effect A' if i == 0 else "")
    if p_main_B[i] < 0.05:
        plt.axvline(x=times[i], ymin=0.07, ymax=0.12, color='blue', alpha=0.5, linestyle='--',
                    label='Main Effect B' if i == 0 else "")
    if p_interaction[i] < 0.05:
        plt.axvline(x=times[i], ymin=0.13, ymax=0.18, color='red', alpha=0.5, linestyle='--',
                    label='Interaction Effect' if i == 0 else "")

plt.title('ERP Analysis with Two-Way ANOVA')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (µV)')
plt.legend(loc='best')
plt.tight_layout()
plt.grid(alpha=0.3)

output_path = os.path.join(output_dir, 'ERP_Two_Way_ANOVA.png')
plt.savefig(output_path, dpi=300)
print(f"Figure saved to {output_path}")
plt.show()

#############################################
# 3. Combined Evoked Data Comparison Plot (Retain This Plot)
#############################################
# Use evoked results from real data
merged_evoke = {
    'Space_Navigation_Reward': R_Case_Merged,
    'Space_Navigation_Punishment': P_Case_Merged,
    'Social_Navigation_Reward': R_Avatar_Merged,
    'Social_Navigation_Punishment': P_Avatar_Merged
}

def plot_evoked_with_anova(merged_evoke, picks, title, output_name):
    # Convert channel labels to indices using channel names from one evoked object
    ch_names = next(iter(merged_evoke.values())).ch_names
    picks_idx = mne.pick_channels(ch_names, picks)
    
    # Get data for selected channels
    data_list = [evk.data[picks_idx] for evk in merged_evoke.values()]
    
    # Reshape data to (n_channels, 1, n_times)
    reshaped_data = [data.reshape(data.shape[0], 1, -1) for data in data_list]
    
    # Combine data (shape: n_channels, 4, n_times)
    data_combine = np.concatenate(reshaped_data, axis=1)
    
    # Perform 2×2 repeated measures ANOVA (for each time point)
    f_main_A, p_main_A = f_mway_rm(data_combine, factor_levels, effects='A')
    f_main_B, p_main_B = f_mway_rm(data_combine, factor_levels, effects='B')
    f_inter, p_interaction = f_mway_rm(data_combine, factor_levels, effects='A:B')

    # FDR correction
    rejects_A, p_main_A = fdr_correction(p_main_A, alpha=0.05)
    rejects_B, p_main_B = fdr_correction(p_main_B, alpha=0.05)
    rejects_inter, p_interaction = fdr_correction(p_interaction, alpha=0.05)

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))
    mne.viz.plot_compare_evokeds(evokeds=merged_evoke, picks=picks_idx, combine='mean', axes=ax, legend='lower right')
    
    # Add vertical lines at significant time points (using ax instead of plt)
    for i in range(len(times)):
        if p_main_A[i] < 0.05:
            ax.axvline(x=times[i], ymin=0.01, ymax=0.06, color='grey', alpha=0.5, linestyle='--',
                       label='Main Effect A' if i == 0 else "")
        if p_main_B[i] < 0.05:
            ax.axvline(x=times[i], ymin=0.07, ymax=0.12, color='blue', alpha=0.5, linestyle='--',
                       label='Main Effect B' if i == 0 else "")
        if p_interaction[i] < 0.05:
            ax.axvline(x=times[i], ymin=0.13, ymax=0.18, color='red', alpha=0.5, linestyle='--',
                       label='Interaction Effect' if i == 0 else "")
    
    fig.suptitle(title)
    fig.savefig(os.path.join(output_dir, output_name), dpi=300)
    print(f"Figure saved to {os.path.join(output_dir, output_name)}")


# Plot ANOVA figures for different regions and save
plot_evoked_with_anova(merged_evoke, picks=['F3', 'F7', 'FC5'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Left Frontal Cortex. Timestamp: 0s~4s',
                       output_name='lFC_ERP_ANOVA.png')

plot_evoked_with_anova(merged_evoke, picks=['F4', 'F8', 'FC6'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Right Frontal Cortex. Timestamp: 0s~4s',
                       output_name='rFC_ERP_ANOVA.png')

plot_evoked_with_anova(merged_evoke, picks=['CP5', 'P7', 'P3'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Left Temporoparietal Junction (lTPJ). Timestamp: 0s~4s',
                       output_name='lTPJ_ERP_ANOVA.png')

plot_evoked_with_anova(merged_evoke, picks=['CP6', 'P4', 'P8'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Right Temporoparietal Junction (rTPJ). Timestamp: 0s~4s',
                       output_name='rTPJ_ERP_ANOVA.png')

plt.show()

In [ ]:
import mne
import os
import numpy as np
import warnings
from mne.stats import fdr_correction, f_mway_rm
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline

#############################################
# 1. Data Preprocessing and Loading
#############################################
# Define the planned time range and baseline correction time range
desired_tmin = 4.2      # Start time of the cropped data
desired_tmax = 6      # End time of the cropped data
baseline_correct_tmin =   # Baseline correction start time
baseline_correct_tmax =   # Baseline correction end time

def read_raw(filename: str) -> mne.io.Raw:
    print(filename)
    if filename.endswith('.fif'):
        raw = mne.io.read_raw_fif(filename, preload=True)
    else:
        raw = mne.io.read_raw_eeglab(filename, preload=True)
    return raw

# Define data types and paths
stimuli_type = ['Reward_Cases', 'Punish_Cases', 'Reward_Avatar', 'Punish_Avatar']
ascent_air = 'Test_Epochs_Leave/'

# Define the path for saving images
output_dir = 'ERP/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# View files in the Reward_Cases folder
for item in os.listdir(os.path.join(ascent_air, 'Reward_Cases')):
    if item.endswith('.fif'):
        print(item, end=' ')
print("\n")

def link_participants_data(data_type: str):
    nlist = []
    nlist_ndarray = []
    folder_path = os.path.join(ascent_air, data_type)
    
    for item in os.listdir(folder_path):
        if item.endswith('.fif'):
            print(f"Processing file: {item}")
            epochs_fname = os.path.join(folder_path, item)
            epochs = mne.read_epochs(epochs_fname, preload=True)
            
            print("Initial epochs data shape:", epochs.get_data().shape)
            epochs.apply_baseline(baseline=(baseline_correct_tmin, baseline_correct_tmax))
            print("Epochs data shape after baseline correction:", epochs.get_data().shape)
            
            evoked = epochs.average()
            print("Evoked data shape after averaging:", evoked.data.shape)
            
            evoked.crop(tmin=desired_tmin, tmax=desired_tmax)
            print("Evoked data shape after cropping:", evoked.data.shape, "\n")
            
            nlist.append(evoked)
            nlist_ndarray.append(evoked.get_data())
    
    return nlist, nlist_ndarray

# Load data for each condition
R_Case_list, R_Case_list_ndarray = link_participants_data('Reward_Cases')
P_Case_list, P_Case_list_ndarray = link_participants_data('Punish_Cases')
R_Avatar_list, R_Avatar_list_ndarray = link_participants_data('Reward_Avatar')
P_Avatar_list, P_Avatar_list_ndarray = link_participants_data('Punish_Avatar')

print("Number of participants per condition:", len(R_Case_list), len(P_Case_list), len(R_Avatar_list), len(P_Avatar_list))
print("Example data shapes:", R_Case_list_ndarray[0].shape, P_Case_list_ndarray[0].shape, 
      R_Avatar_list_ndarray[0].shape, P_Avatar_list_ndarray[0].shape)

# Calculate grand average (group ERP) for each condition
R_Case_Merged = mne.grand_average(R_Case_list)
P_Case_Merged = mne.grand_average(P_Case_list)
R_Avatar_Merged = mne.grand_average(R_Avatar_list)
P_Avatar_Merged = mne.grand_average(P_Avatar_list)

# Use a unified time axis from one evoked object
times = R_Case_Merged.times
print("Unified time axis shape:", times.shape)

#############################################
# 2. Final 2×2 ANOVA Plot: Statistical Analysis and Plotting with Unified Time Axis
#############################################
# Use grand average data
data_A1B1 = R_Case_Merged.data  # Space Navigation Reward
data_A1B2 = P_Case_Merged.data  # Space Navigation Punishment
data_A2B1 = R_Avatar_Merged.data  # Social Navigation Reward
data_A2B2 = P_Avatar_Merged.data  # Social Navigation Punishment

# Reshape data to (n_channels, 1, n_times)
reshaped_A1B1 = data_A1B1.reshape(data_A1B1.shape[0], 1, -1)
reshaped_A1B2 = data_A1B2.reshape(data_A1B2.shape[0], 1, -1)
reshaped_A2B1 = data_A2B1.reshape(data_A2B1.shape[0], 1, -1)
reshaped_A2B2 = data_A2B2.reshape(data_A2B2.shape[0], 1, -1)

# Combine data by condition: resulting shape (n_channels, 4, n_times)
data_combine = np.concatenate((reshaped_A1B1, reshaped_A1B2, reshaped_A2B1, reshaped_A2B2), axis=1)
factor_levels = [2, 2]

# Perform 2×2 repeated measures ANOVA (for each time point)
f_main_A, p_main_A = f_mway_rm(data_combine, factor_levels, effects='A')
f_main_B, p_main_B = f_mway_rm(data_combine, factor_levels, effects='B')
f_inter, p_interaction = f_mway_rm(data_combine, factor_levels, effects='A:B')

# FDR correction
rejects_A, p_main_A = fdr_correction(p_main_A, alpha=0.05)
rejects_B, p_main_B = fdr_correction(p_main_B, alpha=0.05)
rejects_inter, p_interaction = fdr_correction(p_interaction, alpha=0.05)

# Plot the final 2×2 ANOVA figure
plt.figure(figsize=(10, 6))
plt.plot(times, np.average(data_A1B1, axis=0), label='A1B1 (Space Navigation Reward)', color='blue')
plt.plot(times, np.average(data_A1B2, axis=0), label='A1B2 (Space Navigation Punishment)', color='green')
plt.plot(times, np.average(data_A2B1, axis=0), label='A2B1 (Social Navigation Reward)', color='orange')
plt.plot(times, np.average(data_A2B2, axis=0), label='A2B2 (Social Navigation Punishment)', color='purple')

# Add vertical lines at significant time points (different colors for different effects)
for i in range(len(times)):
    if p_main_A[i] < 0.05:
        plt.axvline(x=times[i], ymin=0.01, ymax=0.06, color='grey', alpha=0.5, linestyle='--',
                    label='Main Effect A' if i == 0 else "")
    if p_main_B[i] < 0.05:
        plt.axvline(x=times[i], ymin=0.07, ymax=0.12, color='blue', alpha=0.5, linestyle='--',
                    label='Main Effect B' if i == 0 else "")
    if p_interaction[i] < 0.05:
        plt.axvline(x=times[i], ymin=0.13, ymax=0.18, color='red', alpha=0.5, linestyle='--',
                    label='Interaction Effect' if i == 0 else "")

plt.title('ERP Analysis with Two-Way ANOVA')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (µV)')
plt.legend(loc='best')
plt.tight_layout()
plt.grid(alpha=0.3)

output_path = os.path.join(output_dir, 'ERP_Two_Way_ANOVA.png')
plt.savefig(output_path, dpi=300)
print(f"Figure saved to {output_path}")
plt.show()

#############################################
# 3. Combined Evoked Data Comparison Plot (Retain This Plot)
#############################################
# Use evoked results from real data
merged_evoke = {
    'Space_Navigation_Reward': R_Case_Merged,
    'Space_Navigation_Punishment': P_Case_Merged,
    'Social_Navigation_Reward': R_Avatar_Merged,
    'Social_Navigation_Punishment': P_Avatar_Merged
}

def plot_evoked_with_anova(merged_evoke, picks, title, output_name):
    # Convert channel labels to indices using channel names from one evoked object
    ch_names = next(iter(merged_evoke.values())).ch_names
    picks_idx = mne.pick_channels(ch_names, picks)
    
    # Get data for selected channels
    data_list = [evk.data[picks_idx] for evk in merged_evoke.values()]
    
    # Reshape data to (n_channels, 1, n_times)
    reshaped_data = [data.reshape(data.shape[0], 1, -1) for data in data_list]
    
    # Combine data (shape: n_channels, 4, n_times)
    data_combine = np.concatenate(reshaped_data, axis=1)
    
    # Perform 2×2 repeated measures ANOVA (for each time point)
    f_main_A, p_main_A = f_mway_rm(data_combine, factor_levels, effects='A')
    f_main_B, p_main_B = f_mway_rm(data_combine, factor_levels, effects='B')
    f_inter, p_interaction = f_mway_rm(data_combine, factor_levels, effects='A:B')

    # FDR correction
    rejects_A, p_main_A = fdr_correction(p_main_A, alpha=0.05)
    rejects_B, p_main_B = fdr_correction(p_main_B, alpha=0.05)
    rejects_inter, p_interaction = fdr_correction(p_interaction, alpha=0.05)

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))
    mne.viz.plot_compare_evokeds(evokeds=merged_evoke, picks=picks_idx, combine='mean', axes=ax, legend='lower right')
    
    # Add vertical lines at significant time points (using ax instead of plt)
    for i in range(len(times)):
        if p_main_A[i] < 0.05:
            ax.axvline(x=times[i], ymin=0.01, ymax=0.06, color='grey', alpha=0.5, linestyle='--',
                       label='Main Effect A' if i == 0 else "")
        if p_main_B[i] < 0.05:
            ax.axvline(x=times[i], ymin=0.07, ymax=0.12, color='blue', alpha=0.5, linestyle='--',
                       label='Main Effect B' if i == 0 else "")
        if p_interaction[i] < 0.05:
            ax.axvline(x=times[i], ymin=0.13, ymax=0.18, color='red', alpha=0.5, linestyle='--',
                       label='Interaction Effect' if i == 0 else "")
    
    fig.suptitle(title)
    fig.savefig(os.path.join(output_dir, output_name), dpi=300)
    print(f"Figure saved to {os.path.join(output_dir, output_name)}")


# Plot ANOVA figures for different regions and save
plot_evoked_with_anova(merged_evoke, picks=['F3', 'F7', 'FC5'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Left Frontal Cortex. Timestamp: 4s~6s',
                       output_name='lFC_ERP_ANOVA.png')

plot_evoked_with_anova(merged_evoke, picks=['F4', 'F8', 'FC6'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Right Frontal Cortex. Timestamp: 4s~6s',
                       output_name='rFC_ERP_ANOVA.png')

plot_evoked_with_anova(merged_evoke, picks=['CP5', 'P7', 'P3'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Left Temporoparietal Junction (lTPJ). Timestamp: 4s~6s',
                       output_name='lTPJ_ERP_ANOVA.png')

plot_evoked_with_anova(merged_evoke, picks=['CP6', 'P4', 'P8'], 
                       title='Space-Social Navigation with Reward-Punishment Stimuli: Right Temporoparietal Junction (rTPJ). Timestamp: 4s~6s',
                       output_name='rTPJ_ERP_ANOVA.png')

plt.show()